# AI Agents Crash Course - Part 2: Implementation Notebook

This notebook demonstrates the advanced concepts from Part 2:
- Structured output generation with Pydantic

**Prerequisites:**
1. Install requirements: `pip install -r requirements.txt`
2. Set up `.env` file with your API keys
3. Ensure all project files are in the correct structure

## 1. Setup and Environment

In [1]:
# Install dependencies (uncomment if needed)
# !pip install crewai crewai-tools pydantic python-dotenv requests PyYAML

In [ ]:
import os
from dotenv import load_dotenv
from crewai import LLM

# Load environment variables
load_dotenv()

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## 2. Structured Output with Pydantic

Now let's demonstrate structured output generation using Pydantic schemas.

In [3]:
from crewai import Agent

from pydantic import BaseModel, Field

class EntityRelationEntity(BaseModel):
    entity: str = Field(description="The first entity in the triplet")
    relation: str = Field(description="The relation between the first and second entity")
    entity: str = Field(description="The second entity in the triplet")


In [4]:
from crewai import Agent

agent = Agent(
    role="Senior Linguist",
    goal="Analyse the query and extract entity-relation-entity triplets",
    backstory="You are a senior linguist that is known for your analytical skills.",
    verbose=True,
    llm=llm
)

In [5]:
from crewai import Task

task = Task(
    description="""Analyse the query and return structured JSON
                   output in the form of
                   - entity
                   - relation
                   - entity

                   The query is: {query}
                   """,
    expected_output="""A structured JSON object with the
                       entity-relation-entity triplets""",
    output_pydantic=EntityRelationEntity,
    verbose=True,
    agent=agent
)

In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=[agent],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

response = crew.kickoff(inputs={"query": "Paris is the capital of France."})


In [ ]:
print(response.raw)